# Local constraint application
**Executive summary:** Replace `start_assemble(K,r)` with `KeReAssembler(K,r;ch,apply_zero=true)`.

In some cases, it might be beneficial to apply constraints locally
by using `Ferrite`'s `apply_assemble!`. This is supported by using
`FerriteAssembly`'s `ReAssembler` and `KeReAssembler`.
To demonstrate, let's start by setting up a quick simulation setup

In [1]:
using Ferrite, FerriteAssembly
import FerriteAssembly.ExampleElements: ElasticPlaneStrain
ip = Lagrange{2,RefCube,1}()
grid = generate_grid(Quadrilateral, (3,3))

dh = DofHandler(grid)
add!(dh, :u, 2, ip)
close!(dh)

ch = ConstraintHandler(dh)
add!(ch, Dirichlet(:u, getfaceset(grid, "left"), Returns(zero(Vec{2}))))
add!(ch, Dirichlet(:u, getfaceset(grid, "right"), Returns(1.0), [1,]))
close!(ch)
update!(ch, 0.0)

qr = QuadratureRule{2,RefCube}(2)
cv = CellVectorValues(qr, ip, ip)
m = ElasticPlaneStrain(;E=80e3, ν=0.3)

buffer = setup_domainbuffer(DomainSpec(dh, m, cv))
K = create_sparsity_pattern(dh)
r = zeros(ndofs(dh))
a = zeros(ndofs(dh));

We can now create the special `KeReAssembler`,
which also accepts the constraint handler as input,
in addition to the `apply_zero` keyword that is
forwarded to `Ferrite.apply_assemble!`.

In [2]:
apply!(a, ch)
assembler = KeReAssembler(K, r; ch, apply_zero=true)
work!(assembler, buffer; a=a);

And finally we can solve our problem update

In [3]:
a .-= K\r;

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*